In [ ]:
import numpy as np 
import scipy as sp  
import random     
import bokeh 
import csv
import pandas as pd 
import struct
import os, pickle,sys
from bokeh.plotting import figure, show
from bokeh.io import output_notebook


In [ ]:
#with open('../data/temp-pkl.pkl', 'rb') as pkl_file:
with open('5415172555729652564.pkl', 'rb') as pkl_file:
    df_list = pickle.load(pkl_file)
df = pd.concat(df_list)
df.Timestamp = df.Timestamp - min(df['Timestamp'])

In [ ]:
#single_track = df[df['uniqueId']==df['uniqueId'].unique()[0]]

In [ ]:
#Clean up timestamps so the plot looks better

tf1 = df[df.Timestamp < 5e6]
tf2 = df[(df.Timestamp > 5e6 ) & (df.Timestamp<5.425e6)]
tf3 = df[df.Timestamp > 5.425e6]

tf1_end = max(tf1.Timestamp)
tf2_start = min(tf2.Timestamp)
tf2_end = max(tf2.Timestamp)
tf3_start = min(tf3.Timestamp)
tf3_end = max(tf3.Timestamp)

df['Timestamp'] = df.apply(lambda x: (x['Timestamp']-tf2_start+tf1_end) if ((x['Timestamp'] > 5000000 )& (x['Timestamp'] < 5425000 ) ) else x['Timestamp'],axis=1)
df['Timestamp'] = df.apply(lambda x: (x['Timestamp']-tf3_start+tf1_end+(tf2_end-tf2_start)) if (x['Timestamp'] > 5425000 ) else x['Timestamp'],axis=1)
#df['Timestamp'] = df.apply(lambda x: (x['Timestamp']-tf2_start) if ((x['Timestamp'] > 5000000 )& (x['Timestamp'] < 5425000 ) ) else x['Timestamp'],axis=1)
## Verification plot below:
#output_notebook()
#hist, edges = np.histogram(df['Timestamp'],density=True,bins=5000)
#p = figure(plot_height=600, plot_width=700, title="Timestamp Histogram")
#p.quad(top=hist,bottom=0,left=edges[:-1],right=edges[1:])
#show(p,notebook_handle=True)

Useful columns in this dataset:

'ObjectId'  Unique identifier for each vehicle

'ObjBoxCenter_X' Centre of tracked object in meters offset in the longitudinal axis of the ego vehicle, forward positive

'ObjBoxCenter_Y' Centre of tracked vehicle in meters offset in the transverse axis of the ego vehicle, left positive

'ObjBoxSize_X' Width of tracked vehicle, meters.

'ObjBoxSize_Y'

'AbsVelocity_X' As ego vehicle is stationary, but AbsVelocity_X/y and Relative velocity are equal.

'AbsVelocity_Y'

'AbsVelocity' Combination of X/Y velocity as vector magnitude.

'Classification' Type of vehicle reported by system. Includes:   Bike = 4, Car = 5, Truck = 6

'ClassCertainty' Certainty of above

'ObjBoxOrientation' Orientation of the tracked vehicle in ego frame. Radians. X+ = 0. anti-clockwise.

'Timestamp' Time in seconds since epoch (before the above cell did normalisation)

'origin' The road this vehicle originated from

'destination' The destination of the vehicle

'distance' Distance (meters) travelled from start of intersection. May be negative (approaching intersection)

'distance_to_exit' Distance travelled before the vehicle exits the intersection


All other columns are not relevant, useful, or a artefact of the preprocessor.


In [ ]:

#df.trackedByStationaryModel = (df.trackedByStationaryModel==1)
#df.mobile = (df.mobile==1)
#df['ObjPrediction'] = (df.ObjectPredAge > 0)

In [ ]:
from matplotlib import pyplot as plt
Nlines = 260 
from itertools import permutations     
color_lvl = 8     
rgb = np.array(list(permutations(range(0,256,color_lvl),3))) 
rgb_m = np.array(list(permutations(range(0,256,color_lvl),3)))/255.0

from random import sample
colors = sample(rgb,Nlines) 
colors_m = sample(rgb_m,Nlines)

df['color'] = ["#" + struct.pack('BBB',*colors[i]).encode('hex') for i in df.ObjectId]
df['color_m'] = [tuple(colors_m[i]) for i in df.ObjectId]

In [ ]:
fill_color= [color if flag else None for color, flag in zip(df['color'],df['mobile'])]

In [ ]:
# List of vehicles that travelled above the speed limit:
#list(df[(df.AbsVelocity>50/3.6)].uniqueId.unique())

In [ ]:
output_notebook()
single_track = df[df['uniqueId']==15716]
#print single_track
p = figure(plot_height=600, plot_width=700, title="",x_range=(-40,20),y_range=(-30,30))
#p = figure(plot_height=600, plot_width=700, title="", tools=[hover],x_range=(-100,0),y_range=(-50,50))
#p.image_url(['intersection3.png'],-78.86,40.10,217.62,128.30)
p.image_url(['leith-croydon.png'], x=-15.275, y=-3.1, w=147.45, h=77.0, angle=0, anchor='center', global_alpha=0.7)

p.circle(x=single_track.Object_X, y=single_track.Object_Y, size=4, fill_color="green", line_color="green")
show(p,notebook_handle=True)

In [ ]:
#Here is a plot of the first 1000 tracks in the dataset. Any more and the plotter has trouble keeping up.

output_notebook()
single_track = df[df['uniqueId']==4856]
p = figure(plot_height=400, plot_width=400, title="",x_range=(-35,10),y_range=(-30,15))
p.image_url(['leith-croydon.png'], x=-15.275, y=-3.1, w=147.45, h=77.0, angle=0, anchor='center', global_alpha=0.7)
line_idx = 0
for track_idx in df['uniqueId'].unique():
    single_track = df[df['uniqueId']==track_idx]
    #print single_track

    #p = figure(plot_height=600, plot_width=700, title="", tools=[hover],x_range=(-100,0),y_range=(-50,50))
    #p.image_url(['intersection3.png'],-78.86,40.10,217.62,128.30)
    p.line(x=single_track.Object_X, y=single_track.Object_Y, line_width=2, line_color=single_track['color'][0])
    #print single_track['color']
    line_idx +=1
    if line_idx > 1000:
        break
show(p,notebook_handle=True)

In [ ]:
#A plot of the speed profile of one of the above tracks

output_notebook()
p = figure(plot_height=600, plot_width=700, title="Single_track")
p.line(x=single_track.distance, y=single_track.AbsVelocity*3.6, line_width=4, line_color='blue')
show(p,notebook_handle=True)


In [ ]:
# The next few cells will give the speed and heading plots over distance, which are figures in the paper.
# To do this, we need to align the tracks by distance travelled. Its time consuming and unoptimized.
# Feel free to skip to the last two cells instead.
record_dict = {}
df.origin.unique()
for origin in df.origin.unique():
    for destination in df.destination.unique():
        if origin in destination:
            continue
        dict_key = str(origin + '-' + destination)
        print origin + '-' + destination
        class_df = df[(df['origin']==origin) & (df['destination']==destination)]
        class_df = class_df.sort_values('distance',ascending=False)
        max_dis = int(max(class_df.distance.unique())) # max_dis = 60 # 
        min_dis = int(min(class_df.distance.unique())) #min_dis = -10 #
        n_vehicles = len(class_df.uniqueId.unique())
        vehicle_cnt = 0
        records = []
        for uniqueId in class_df.uniqueId.unique():
            vehicle_cnt+=1
            sys.stdout.write("\rWorking on vehicle %d of %d" % (vehicle_cnt, n_vehicles))
            sys.stdout.flush()
            vehicle_df = class_df[class_df['uniqueId']==uniqueId]
            vehicle_df = vehicle_df.sort_values('distance',ascending=False)
            for d_thresh in np.arange(min_dis,max_dis,0.5):
                #print "distance: " + str(d_thresh)
                
                #Find the closest but not over point, add to a thing.
                tp = vehicle_df[vehicle_df['distance']<d_thresh]
                try:
                    record = tp.iloc[range(1)] #
                except IndexError:
                    # This track does not exist at said distance. Continue
                    continue
                record = record.assign(d_thresh=np.repeat(d_thresh, len(record)))
                # if it is not within 1 meter, there is no data, and we are redundantly appending the last results
                if record.iloc[0].distance > (d_thresh -1):
                    records.append(record)
        print " Concatenating Records..."
        records = pd.concat(records)
        print "done"
        record_dict[dict_key] = records
        

In [ ]:
import scipy.stats
track_classes = list(record_dict.keys())
track_colors = ['Cyan','DarkBlue','Chartreuse', 'SeaGreen','Fuchsia','FireBrick']
origin_destination = 'east-north'
speeds_dt = {}
speed_stds_dt={}
headings_dt = {}
heading_stds_dt = {}
distances_dt = {}
population_dt = {}
for track_class, color in zip(track_classes,track_colors):
    tf = record_dict[track_class]
    distances = np.sort(tf['d_thresh'].unique())
    distances_dt[track_class] = distances
    speeds = []
    speed_stds = []
    headings = []
    heading_stds = []
    track_pop = []
    for d_thresh in distances:
        df_at_dis = tf[tf['d_thresh']==d_thresh]
        #print df_at_dis.columns
        #Collect various data for plot here. I am going to do speed and heading
        speeds.append(np.average(df_at_dis['AbsVelocity'])*3.6)
        speed_stds.append(np.std(df_at_dis['AbsVelocity']*3.6))
        #headings.append(np.average(df_at_dis['ObjBoxOrientation']))
        heading = np.arctan2(np.sum(np.sin(df_at_dis['ObjBoxOrientation'])),
                                 np.sum(np.cos(df_at_dis['ObjBoxOrientation'])))
        # Clean up graph as its circular data
        heading_std = scipy.stats.circstd(df_at_dis['ObjBoxOrientation'],low=-np.pi,high=np.pi)
        if 'south' in df_at_dis['origin'].unique():
            if heading < 0:
                heading += 3.14159*2
        if 'east-north' in track_class:
            if heading > 0:
                heading -= 3.14159*2
        headings.append(heading)
        heading_stds.append(heading_std)
        track_pop.append(len(df_at_dis))
    heading_stds_dt[track_class] = heading_stds    
    speeds_dt[track_class] = speeds
    speed_stds_dt[track_class] = speed_stds
    headings_dt[track_class] = headings
    population_dt[track_class] = track_pop

        

In [ ]:
for track_class, color in zip(track_classes,track_colors):
    tf = record_dict[track_class]
    heading = np.arctan2(np.sum(np.sin(tf['ObjBoxOrientation'])),
                                 np.sum(np.cos(tf['ObjBoxOrientation'])))
    print track_class + " Av Speed: %.2f Std: %.2f \
    max: %.2f heading: %.2f std: %.2f" % (np.average(tf.AbsVelocity*3.6),
                                        np.std(tf.AbsVelocity*3.6),
                                        max(tf.AbsVelocity*3.6),
                                        heading,
                                        np.average(heading_stds_dt[track_class]))

In [ ]:
#PLOT STUFF FROM record_dict
output_notebook()
p = figure(plot_height=600, plot_width=700, title="Speeds")
for track_class, color in zip(track_classes,track_colors):
    speed = np.array(speeds_dt[track_class])
    speedstd = np.array(speed_stds_dt[track_class])
    p.line(x=distances_dt[track_class], y=speeds_dt[track_class],legend=track_class, line_width=4, line_color=color)
    p.line(x=distances_dt[track_class], y=speed+speedstd, line_width=1, line_color=color, line_dash='dashed')
    p.line(x=distances_dt[track_class], y=speed-speedstd, line_width=1, line_color=color, line_dash='dashed')
show(p,notebook_handle=True)


In [ ]:
output_notebook()
p = figure(plot_height=600, plot_width=700, title="Headings")
for track_class, color in zip(track_classes,track_colors):
    heading = np.array(headings_dt[track_class])
    headingstd = np.array(heading_stds_dt[track_class])
    p.line(x=distances_dt[track_class], y=headings_dt[track_class], legend=track_class, line_width=4, line_color=color)
    p.line(x=distances_dt[track_class], y=heading + heading_std, line_width=2,
           line_color=color, line_dash='dashed')
    p.line(x=distances_dt[track_class], y=heading - heading_std, line_width=2,
           line_color=color, line_dash='dashed')
show(p,notebook_handle=True)

In [ ]:
output_notebook()
p = figure(plot_height=600, plot_width=700, title="Population")
for track_class, color in zip(track_classes,track_colors):
    p.line(x=distances_dt[track_class], y=population_dt[track_class], legend=track_class, line_width=4, line_color=color)
show(p,notebook_handle=True)

In [ ]:
# TIMESTAMP EXPLORER
# This graph is an interactive display of the vehicles in a top down view. The sliders below allow for control of
# the current timestamp, and the time window in which the vehicles may be visible. Reduce this to ~5 to view one
#car at a time
# It can be quite resource intensive, so be patient.
# A coloured dot means that was a tracked datapoint. A white dot means the tracker could not see the target, and 
# was extrapolating the vehicle's position
# The 
from bokeh.plotting import figure
from bokeh.layouts import layout, widgetbox
from bokeh.models import ColumnDataSource, HoverTool, Div
from bokeh.models.widgets import Slider, Select, TextInput
from bokeh.io import curdoc, push_notebook
from ipywidgets import interact
import ipywidgets

output_notebook()

time_max = max(df['Timestamp'])
time_min = min(df['Timestamp'])

time_slider = Slider(start=time_min,end=time_max,value=time_min,step=1,title="Time")
source_rect = ColumnDataSource(data=dict(x=[], y=[], width=[], height=[],angle=[],RefPointLocation=[],
                                    color=[], title=[], year=[], revenue=[], alpha=[]))
source_circle = ColumnDataSource(data=dict(x=[], y=[], line_color=[], fill_color=[],RefPointLocation=[],
                                           title=[], year=[], revenue=[], alpha=[]))
source_crossH = ColumnDataSource(data=dict(x=[], y=[], width=[], height=[],angle=[],RefPointLocation=[],
                                    color=[], title=[], year=[], revenue=[], alpha=[]))
source_crossW = ColumnDataSource(data=dict(x=[], y=[], width=[], height=[],angle=[],RefPointLocation=[],
                                    color=[], title=[], year=[], revenue=[], alpha=[]))
source_ref_pt = ColumnDataSource(data=dict(x=[], y=[], line_color=[], fill_color=[], RefPointLocation=[],
                                           title=[], year=[], revenue=[], alpha=[]))
source_seg = ColumnDataSource(data=dict(x0=[], y0=[], x1=[], y1=[], line_color=[], fill_color=[], RefPointLocation=[],
                                           title=[], year=[], revenue=[], alpha=[]))
source_seg_accel = ColumnDataSource(data=dict(x0=[], y0=[], x1=[], y1=[], line_color=[], fill_color=[], RefPointLocation=[],
                                           title=[], year=[], revenue=[], alpha=[]))
#Sliders:
slider_time = ipywidgets.FloatSlider(
    #value = (time_max+time_min)/2, min=time_min, max=time_max, step = 0.1, description="Time",slider_color="red")
    value = time_max, min=time_min, max=time_max, step = 0.1, description="Time",slider_color="red")
window_slider = ipywidgets.FloatSlider(
    #value = 5, min=1, max=time_max, step = 0.1, description="Time Window",slider_color="blue")
    value = time_max, min=1, max=time_max, step = 0.1, description="Time Window",slider_color="blue")
min_conf_slider = ipywidgets.FloatSlider(
    value = 5, min=0, max=100, step = 1, description="Min Classification Score",slider_color="green")


hover = HoverTool(tooltips=[
    ("ObjectId", "@ObjectId"),
    ("Object_Y","@ObjBoxCenter_Y"),
    ("Object_X","@ObjBoxCenter_X"),
    ("Vel","@AbsVelocity"),
    #("$", "@revenue")
])
# tools=[hover,'pan','wheel_zoom','box_zoom','reset','resize'],
p = figure(plot_height=600, plot_width=700, title="", x_range=(-40,20),y_range=(-30,30))
#p = figure(plot_height=600, plot_width=700, title="", tools=[hover],x_range=(-100,0),y_range=(-50,50))
#p.image_url(['intersection3.png'],-78.86,40.10,217.62,128.30)
p.image_url(['leith-croydon.png'], x=-15.275, y=-3.1, w=147.45, h=77.0, angle=0, anchor='center', global_alpha=0.7)
p.rect(x="x", y="y", width="width",height="height", angle="angle",source=source_crossH, fill_color="color",
       line_color="color")
p.circle(x="x", y="y", source=source_circle, size=4, fill_color="fill_color", line_color="line_color")
p.circle(x="x", y="y", source=source_ref_pt, size=3, fill_color="black", line_color="black")
p.rect(x="x", y="y", width="width",height="height", angle="angle",source=source_rect, fill_color=None,
       line_color="color")
p.rect(x="x", y="y", width="width",height="height", angle="angle",source=source_crossW, fill_color="color",
       line_color="color")
p.segment(x0='x0',y0='y0',x1='x1',y1='y1',line_width=2,line_color='orange',source=source_seg)
p.segment(x0='x0',y0='y0',x1='x1',y1='y1',line_width=2,line_color='pink',source=source_seg_accel)


show(p,notebook_handle=True)

def select_data(time_val, time_window, min_conf):
    #time_val = time_slider.value
    #t_min = time_val-(float(time_window)/2.0)
    #t_max = time_val+(float(time_window)/2.0)
    t_min = time_val-time_window
    t_max = time_val

    selected = df[
        (df.Timestamp > t_min)
        &
        (df.Timestamp < t_max)
        &
        (df.Classification >3 )
        &
        (df.ClassCertainty > min_conf)
    ]
    
    return_df = pd.DataFrame()
    
    for obj_id in selected.ObjectId.unique():
        object_df = selected[selected.ObjectId==obj_id].sort_values(['Timestamp'],ascending=False)
        selected_record=object_df.iloc[0].copy()
        if len(object_df) > 1:
            selected_record['AbsAccel_X'] = object_df['AbsVelocity_X'].iloc[0] - object_df['AbsVelocity_X'].iloc[1]
            selected_record['AbsAccel_Y'] = object_df['AbsVelocity_Y'].iloc[0] - object_df['AbsVelocity_Y'].iloc[1]
        else:
            selected_record['AbsAccel_X'] = 0
            selected_record['AbsAccel_Y'] = 0
        return_df = return_df.append(selected_record)
        
    # Only draw box if car was seen in last 0.5 seconds. 
    # I really only want if it was seen at THIS timestep, but this is close enough
    recent_only_df = return_df[return_df['Timestamp']> (time_val-0.2)]
    #print "Selected data length: " + str(len(selected))
    #print recent_only_df
    return recent_only_df, selected #selected

def update(time_value=slider_time,time_window=window_slider,min_conf=min_conf_slider):
    df_rect, df_circle = select_data(time_value,time_window,min_conf)
    
    source_rect.data = dict(
        x=df_rect['ObjBoxCenter_X'],
        y=df_rect['ObjBoxCenter_Y'],
        width=df_rect['ObjBoxSize_X'],
        height=df_rect['ObjBoxSize_Y'],
        angle=df_rect['ObjBoxOrientation'],
        color=df_rect['color'],
        ObjectId=df_rect['ObjectId'],
        RefPointLocation = df_rect['RefPointLocation'],
        AbsVelocity = df_rect['AbsVelocity'],
        Timestamp=df_rect['Timestamp']
    )
    source_ref_pt.data = dict(
        x=df_rect['RefPointCoords_X'],
        y=df_rect['RefPointCoords_Y'],
        ObjectId=df_rect['ObjectId'],
        RefPointLocation = df_rect['RefPointLocation'],
        AbsVelocity = df_rect['AbsVelocity'],
        Timestamp=df_rect['Timestamp']
    )
    source_seg.data = dict(
        x0=df_rect['ObjBoxCenter_X'],
        y0=df_rect['ObjBoxCenter_Y'],
        x1=df_rect['ObjBoxCenter_X'].add(df_rect['AbsVelocity_X']),
        y1=df_rect['ObjBoxCenter_Y'].add(df_rect['AbsVelocity_Y']),
        ObjectId=df_rect['ObjectId'],
        RefPointLocation = df_rect['RefPointLocation'],
        AbsVelocity = df_rect['AbsVelocity'],
        Timestamp=df_rect['Timestamp']
    )
    source_seg_accel.data = dict(
        x0=df_rect['ObjBoxCenter_X'],
        y0=df_rect['ObjBoxCenter_Y'],
        x1=df_rect['ObjBoxCenter_X'].add(10*df_rect['AbsAccel_X']),
        y1=df_rect['ObjBoxCenter_Y'].add(10*df_rect['AbsAccel_Y']),
        ObjectId=df_rect['ObjectId'],
        #RefPointLocation = df_rect['RefPointLocation'],
        AbsVelocity = df_rect['AbsVelocity'],
        Timestamp=df_rect['Timestamp']
    )
    source_circle.data = dict(
        x=df_circle['ObjBoxCenter_X'],
        y=df_circle['ObjBoxCenter_Y'],
        fill_color= [color if not flag else "#FFFFFF" for color,flag in zip(df_circle['color'],df_circle['ObjPrediction'])],
        line_color=df_circle['color'],
        ObjectId=df_circle['ObjectId'],
        Timestamp=df_circle['Timestamp']
    )
    source_crossW.data = dict(
        x=df_rect['ObjBoxCenter_X'],
        y=df_rect['ObjBoxCenter_Y'],
        width=df_rect['ObjBoxCenterSigma_X'],
        height=[0.1]*len(df_rect),
        angle=df_rect['ObjBoxOrientation'],
        color=df_rect['color'],
        ObjectId=df_rect['ObjectId'],
        RefPointLocation = df_rect['RefPointLocation'],
        AbsVelocity = df_rect['AbsVelocity'],
        Timestamp=df_rect['Timestamp']
    )
    #'ObjBoxCenterSigma_Y'
    source_crossH.data = dict(
        x=df_rect['ObjBoxCenter_X'],
        y=df_rect['ObjBoxCenter_Y'],
        width=[0.1]*len(df_rect),
        height=df_rect['ObjBoxCenterSigma_Y'],
        angle=df_rect['ObjBoxOrientation'],
        color=df_rect['color'],
        ObjectId=df_rect['ObjectId'],
        #RefPointLocation = df_rect['RefPointLocation'],
        AbsVelocity = df_rect['AbsVelocity'],
        Timestamp=df_rect['Timestamp']
    )

    push_notebook()


update(time_max,1,5)  # initial load of the data



In [ ]:
#If this fails run $ jupyter nbextension enable --py --sys-prefix widgetsnbextension

interact(update,time_val=slider_time,time_window=window_slider,min_conf=min_conf_slider)
